## Neural network (RNN with LSTM layer)

Why a neural network with LSTM layer?

We are still on our binary classification of tweets/comments, the label 0 is for "sarcasm" and the label 1 means "cyberbullying".

A neural network, unlike the previous Naive Bayes model that we have tried out, will make use of the context (as opposed to dealing with individual words).
And a Long Short Term Memory layer means the model can handle “long-term dependencies” (which a classical RNN cannot handle).
LSTM supposedly also solves the problem of vanishing gradient.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras import layers, Sequential
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import gensim.downloader as api

2023-02-02 12:36:36.333538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 12:36:36.443208: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-02 12:36:36.447280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 12:36:36.447292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
data = pd.read_csv('merged_all_modified_csv.csv')
X = data['comment_cleaned_lower']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [16]:
X_train.shape

(29751,)

In [30]:
#Train a word2vec model on our sentences
word2vec = Word2Vec(sentences=[str(x) for x in X_train], vector_size=30, window =2, min_count=5)
#word2vec = Word2Vec(sentences=str(X_train), vector_size=30, window =2, min_count=5)
print(word2vec)

Word2Vec<vocab=124, vector_size=30, alpha=0.025>


In [31]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = [] 
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)    
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, [str(x) for x in X_train])
X_test_embed = embedding(word2vec, [str(x) for x in X_test])

# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [32]:
X_train_pad.shape, X_test_pad.shape

((29751, 200, 30), (12751, 200, 30))

In [33]:
#checking our X_train_pad and X_test_pad, they should be np arrays, 3-dim,  
#last dimension must be of the size of the word2vec embedding space, and 1st dim must be of size of X_train and X_test

for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

## Baseline accuracy

In [34]:
baseline_accuracy = y_train.sum() / y_train.shape[0]
baseline_accuracy

0.7490168397700918

## RNN model, without transfer learning

In [35]:
vocab_size = word2vec.wv.vectors.shape[0]

In [36]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation="tanh"))
    model.add(layers.Dense(10, activation = 'relu'))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    return model

model = init_model()

In [37]:
callback = EarlyStopping(monitor='accuracy', patience=3)
model.fit(X_train_pad, y_train, epochs=10, batch_size=128, callbacks=[callback], verbose=1)

Epoch 1/10
233/233 [==============================] - 87s 351ms/step - loss: 0.5372 - accuracy: 0.7471
Epoch 2/10
233/233 [==============================] - 83s 357ms/step - loss: 0.4463 - accuracy: 0.7887
Epoch 3/10
233/233 [==============================] - 87s 372ms/step - loss: 0.4370 - accuracy: 0.8163
Epoch 4/10
233/233 [==============================] - 73s 315ms/step - loss: 0.3956 - accuracy: 0.8379
Epoch 5/10
233/233 [==============================] - 77s 330ms/step - loss: 0.3519 - accuracy: 0.8533
Epoch 6/10
233/233 [==============================] - 77s 331ms/step - loss: 0.3398 - accuracy: 0.8659
Epoch 7/10
233/233 [==============================] - 32s 138ms/step - loss: 0.3158 - accuracy: 0.8687
Epoch 8/10
233/233 [==============================] - 38s 165ms/step - loss: 0.3033 - accuracy: 0.8749
Epoch 9/10
233/233 [==============================] - 74s 316ms/step - loss: 0.3222 - accuracy: 0.8707
Epoch 10/10
233/233 [==============================] - 73s 314ms/step - l

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 200, 30)           0         
                                                                 
 lstm_1 (LSTM)               (None, 20)                4080      
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,301
Trainable params: 4,301
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.predict(X_test_pad)

399/399 [==============================] - 25s 59ms/step


array([[0.97180146],
       [0.95489734],
       [0.9362625 ],
       ...,
       [0.9579762 ],
       [0.9467767 ],
       [0.9647433 ]], dtype=float32)

In [ ]:
model.evaluate(X_test_pad, y_test)

## Same model, this time pretrained on much larger (and similar) dataset : Glove
transfer learning


In [40]:
#print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [41]:
word2vec_transfer = api.load('glove-twitter-200')

In [42]:
embedding_size_transfer = word2vec_transfer.vector_size
vocab_size_transfer = word2vec_transfer.vectors.shape[0]

In [44]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []  
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)   
    return embed

# Embed the training and test sentences
X_train_embed_transfer = embedding(word2vec_transfer, [str(x) for x in X_train])
X_test_embed_transfer = embedding(word2vec_transfer, [str(x) for x in X_test])

In [45]:
#padding
X_train_pad_transfer = pad_sequences(X_train_embed_transfer, dtype='float32', padding='post', maxlen=200)
X_test_pad_transfer = pad_sequences(X_test_embed_transfer, dtype='float32', padding='post', maxlen=200)

In [46]:
model_transfer = init_model()

In [47]:
model_transfer.fit(X_train_pad_transfer, y_train, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
233/233 [==============================] - 81s 332ms/step - loss: 0.5343 - accuracy: 0.7447
Epoch 2/100
233/233 [==============================] - 75s 324ms/step - loss: 0.4814 - accuracy: 0.7645
Epoch 3/100
233/233 [==============================] - 77s 332ms/step - loss: 0.4202 - accuracy: 0.8061
Epoch 4/100
233/233 [==============================] - 76s 327ms/step - loss: 0.3711 - accuracy: 0.8325
Epoch 5/100
233/233 [==============================] - 76s 325ms/step - loss: 0.3329 - accuracy: 0.8525
Epoch 6/100
233/233 [==============================] - 76s 325ms/step - loss: 0.3081 - accuracy: 0.8633
Epoch 7/100
233/233 [==============================] - 84s 360ms/step - loss: 0.2894 - accuracy: 0.8726
Epoch 8/100
233/233 [==============================] - 76s 326ms/step - loss: 0.2756 - accuracy: 0.8803
Epoch 9/100
233/233 [==============================] - 76s 325ms/step - loss: 0.2609 - accuracy: 0.8879
Epoch 10/100
233/233 [==============================] - 76s 326m

233/233 [==============================] - 76s 325ms/step - loss: 0.0639 - accuracy: 0.9774
Epoch 80/100
233/233 [==============================] - 76s 326ms/step - loss: 0.0628 - accuracy: 0.9778
Epoch 81/100
233/233 [==============================] - 76s 328ms/step - loss: 0.0614 - accuracy: 0.9782
Epoch 82/100
233/233 [==============================] - 76s 325ms/step - loss: 0.0614 - accuracy: 0.9786
Epoch 83/100
233/233 [==============================] - 77s 329ms/step - loss: 0.0609 - accuracy: 0.9793
Epoch 84/100
233/233 [==============================] - 77s 329ms/step - loss: 0.0591 - accuracy: 0.9793
Epoch 85/100
233/233 [==============================] - 75s 321ms/step - loss: 0.0596 - accuracy: 0.9787
Epoch 86/100
233/233 [==============================] - 79s 338ms/step - loss: 0.0588 - accuracy: 0.9792
Epoch 87/100
233/233 [==============================] - 83s 356ms/step - loss: 0.0579 - accuracy: 0.9796
Epoch 88/100
233/233 [==============================] - 84s 360ms/st

In [48]:
model_transfer.predict(X_test_pad_transfer)

399/399 [==============================] - 35s 62ms/step


array([[9.9960357e-01],
       [9.9983239e-01],
       [9.9295008e-01],
       ...,
       [9.9990988e-01],
       [4.3896434e-04],
       [9.9986994e-01]], dtype=float32)

In [49]:
res = model_transfer.evaluate(X_test_pad_transfer, y_test, verbose=2)

399/399 - 22s - loss: 0.1285 - accuracy: 0.9602 - 22s/epoch - 56ms/step
